In [278]:
from ast import literal_eval
import pandas as pd
import numpy as np
import lucem_illud_2020

import scipy
import sklearn_extra
import sklearn
import sklearn.feature_extraction.text
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.datasets
import sklearn.cluster
import sklearn.decomposition
import sklearn.metrics
import sklearn.mixture

import pyLDAvis
import pyLDAvis.gensim
import gensim
import networkx as nx
from gensim.models import ldaseqmodel
from gensim.matutils import kullback_leibler
from gensim import models

import matplotlib.pyplot as plt
import matplotlib.cm
import seaborn as sns
import altair as alt

from source import helper_functions as hf, cluster_fns
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
tal_df = hf.load_df('total_trans_normalized.csv')

In [8]:
tal_df = hf.split_five_years(tal_df)
tal_df.head()

,ep_num,ep_title,year,url,act_name,text,tokenized_sents,no_lemma_normalized_sents,normalized_sents,tokenized_text,normalized_tokens,no_lemma_normalized_tokens,five_yr_group
0,1,New Beginnings,1995,https://www.thisamericanlife.org/1/transcript,Act Four: Act Four,This is Your Radio Playhouse. I'm Ira Glass. O...,"[[This, is, Your, Radio, Playhouse], [I, 'm, I...","[[radio, playhouse], [m, ira, glass], [ok, rig...","[[radio, playhouse], [be, ira, glass], [okay, ...","[This, is, Your, Radio, Playhouse, I, 'm, Ira,...","[radio, playhouse, be, ira, glass, okay, right...","[radio, playhouse, m, ira, glass, ok, right, r...",0.0
1,1,New Beginnings,1995,https://www.thisamericanlife.org/1/transcript,Act Three: Act Three,"Well, next on our little playhouse stage, we h...","[[Well, next, on, our, little, playhouse, stag...","[[little, playhouse, stage, mr, lawrence, steg...","[[little, playhouse, stage, mr, lawrence, steg...","[Well, next, on, our, little, playhouse, stage...","[little, playhouse, stage, mr, lawrence, stege...","[little, playhouse, stage, mr, lawrence, stege...",0.0
2,1,New Beginnings,1995,https://www.thisamericanlife.org/1/transcript,Act Two: Act Two,"Good morning. Glass, Jacobson & Associates.Hey...","[[Good, morning], [Glass, Jacobson, Associates...","[[good, morning], [glass, jacobson, associates...","[[good, morning], [glass, jacobson, associate]...","[Good, morning, Glass, Jacobson, Associates, H...","[good, morning, glass, jacobson, associate, he...","[good, morning, glass, jacobson, associates, h...",0.0
3,1,New Beginnings,1995,https://www.thisamericanlife.org/1/transcript,Act One: Act One,"All right, Your Radio Playhouse. All right. I'...","[[All, right, Your, Radio, Playhouse], [All, r...","[[right, radio, playhouse], [right], [m, makin...","[[right, radio, playhouse], [right], [be, make...","[All, right, Your, Radio, Playhouse, All, righ...","[right, radio, playhouse, right, be, make, eye...","[right, radio, playhouse, right, m, making, ey...",0.0
4,1,New Beginnings,1995,https://www.thisamericanlife.org/1/transcript,Prologue,"Joe Franklin?I'm ready.It's Ira Glass here.Oh,...","[[Joe, Franklin?I'm, ready], [It, 's, Ira, Gla...","[[joe, franklin?i'm, ready], [ira, glass], [oh...","[[joe, franklin?i'm, ready], [ira, glass], [oh...","[Joe, Franklin?I'm, ready, It, 's, Ira, Glass,...","[joe, franklin?i'm, ready, ira, glass, oh, emc...","[joe, franklin?i'm, ready, ira, glass, oh, emc...",0.0


In [86]:
by_period = hf.agg_text(tal_df, 'five_yr_group', False)
by_period.head()

,five_yr_group,text,tokenized_sents,no_lemma_normalized_sents,normalized_sents,tokenized_text,normalized_tokens,no_lemma_normalized_tokens,reduced_tokens
0,0.0,This is Your Radio Playhouse. I'm Ira Glass. O...,"[[This, is, Your, Radio, Playhouse], [I, 'm, I...","[[radio, playhouse], [m, ira, glass], [ok, rig...","[[radio, playhouse], [be, ira, glass], [okay, ...","[This, is, Your, Radio, Playhouse, I, 'm, Ira,...","[radio, playhouse, be, ira, glass, okay, right...","[radio, playhouse, m, ira, glass, ok, right, r...","[playhouse, ryder, graterford, graterford, gra..."
1,1.0,It's This American Life. I'm Ira Glass. Most w...,"[[It, 's, This, American, Life], [I, 'm, Ira, ...","[[american, life], [m, ira, glass], [weeks, pr...","[[american, life], [be, ira, glass], [week, pr...","[It, 's, This, American, Life, I, 'm, Ira, Gla...","[american, life, be, ira, glass, week, program...","[american, life, m, ira, glass, weeks, program...","[redding, lac, meister, bieber, bieber, bieber..."
2,2.0,"Act Three, Babies Buying Babies. Elna Baker a...","[[Act, Three, Babies, Buying, Babies], [Elna, ...","[[act, babies, buying, babies], [elna, baker, ...","[[act, baby, buy, baby], [elna, baker, want, a...","[Act, Three, Babies, Buying, Babies, Elna, Bak...","[act, baby, buy, baby, elna, baker, want, actr...","[act, babies, buying, babies, elna, baker, wan...","[elna, elna, middleton, middleton, middleton, ..."
3,3.0,"Act Four, The Deepest, Darkest Open Secret. Mo...","[[Act, Four, The, Deepest, Darkest, Open, Secr...","[[act, deepest, darkest, open, secret], [years...","[[act, deep, dark, open, secret], [year, ago, ...","[Act, Four, The, Deepest, Darkest, Open, Secre...","[act, deep, dark, open, secret, year, ago, sol...","[act, deepest, darkest, open, secret, years, a...","[sonari, sonari, sonari, sonari, sonari, sonar..."


## Find Topics by Year & Apply to 5 Years

### Prologues

In [ ]:
# First get topics from prologues

### Episodes

In [63]:
# topics from episodes
ep_df = hf.agg_text(tal_df, 'ep_title', True)
ep_df.head()

,ep_title,ep_num,year,url,text,tokenized_sents,no_lemma_normalized_sents,normalized_sents,tokenized_text,normalized_tokens,no_lemma_normalized_tokens
0,A Better Mousetrap,311,2006,https://www.thisamericanlife.org/311/transcript,Andy's grandfather ran the business. Then his ...,"[[Andy, 's, grandfather, ran, the, business], ...","[[andy, grandfather, ran, business], [father, ...","[[andy, grandfather, run, business], [father, ...","[Andy, 's, grandfather, ran, the, business, Th...","[andy, grandfather, run, business, father, run...","[andy, grandfather, ran, business, father, ran..."
1,A Better Mousetrap 2008,366,2008,https://www.thisamericanlife.org/366/transcript,"Act Four, ""The Not-For-Profit Motive."" In resp...","[[Act, Four, The, Not, For, Profit, Motive], [...","[[act, profit, motive], [], [responding, curre...","[[act, profit, motive], [], [respond, current,...","[Act, Four, The, Not, For, Profit, Motive, In,...","[act, profit, motive, respond, current, financ...","[act, profit, motive, responding, current, fin..."
2,A Front,540,2014,https://www.thisamericanlife.org/540/transcript,"Act One, WTF, ATF? So a funny thing happened a...","[[Act, One, WTF, ATF], [So, a, funny, thing, h...","[[act, wtf, atf], [funny, thing, happened, joh...","[[act, wtf, atf], [funny, thing, happen, john,...","[Act, One, WTF, ATF, So, a, funny, thing, happ...","[act, wtf, atf, funny, thing, happen, john, ra...","[act, wtf, atf, funny, thing, happened, john, ..."
3,A House Divided,439,2011,https://www.thisamericanlife.org/439/transcript,It's This American Life. I'm Ira Glass. Each w...,"[[It, 's, This, American, Life], [I, 'm, Ira, ...","[[american, life], [m, ira, glass], [week, cou...","[[american, life], [be, ira, glass], [week, co...","[It, 's, This, American, Life, I, 'm, Ira, Gla...","[american, life, be, ira, glass, week, course,...","[american, life, m, ira, glass, week, course, ..."
4,A Little Bit of Knowledge,293,2005,https://www.thisamericanlife.org/293/transcript,It's This American Life. I'm Ira Glass. Each w...,"[[It, 's, This, American, Life], [I, 'm, Ira, ...","[[american, life], [m, ira, glass], [week, pro...","[[american, life], [be, ira, glass], [week, pr...","[It, 's, This, American, Life, I, 'm, Ira, Gla...","[american, life, be, ira, glass, week, program...","[american, life, m, ira, glass, week, program,..."


In [253]:
min_df = 0.2
max_df = 0.8
new_models = []
for num in [4, 5, 6, 7]:
    ldamodel, output = cluster_fns.new_model_fn(ep_df, 'text', num, min_df, max_df)
    print(cluster_fns.make_topic_df(ldamodel))
    new_models.append((num, ldamodel))
    print()

0.2 0.8
[(0, '0.004*"book" + 0.003*"girl" + 0.003*"women"'), (1, '0.007*"police" + 0.004*"state" + 0.004*"court"'), (2, '0.009*"speaking" + 0.006*"president" + 0.005*"government"'), (3, '0.009*"mother" + 0.008*"dad" + 0.007*"mom"')]
  Topic_0    Topic_1     Topic_2   Topic_3
0    book     police    speaking    mother
1    girl      state   president       dad
2   women      court  government       mom
3     000     father       white    father
4    play  christmas       state   parents
5  street        war         war      book
6    york     mother     spanish       kid
7  father        mom       black     white
8    song       drug         000     black
9  mother       town        town  children

0.2 0.8
[(0, '0.004*"women" + 0.004*"girl" + 0.004*"book"'), (1, '0.006*"police" + 0.005*"father" + 0.004*"christmas"'), (2, '0.014*"speaking" + 0.008*"spanish" + 0.004*"dad"'), (3, '0.010*"mother" + 0.009*"dad" + 0.008*"mom"'), (4, '0.006*"state" + 0.006*"government" + 0.005*"white"')]
  Top

In [254]:
min_df = 5
max_df = 0.9
new_models2 = []
for num in [4, 5, 6, 7]:
    ldamodel, output = cluster_fns.new_model_fn(ep_df, 'text', num, min_df, max_df)
    print(cluster_fns.make_topic_df(ldamodel))
    new_models2.append((num, ldamodel))
    print()

5 0.9
[(0, '0.004*"kids" + 0.003*"love" + 0.002*"city"'), (1, '0.003*"car" + 0.003*"mother" + 0.003*"father"'), (2, '0.003*"country" + 0.003*"president" + 0.002*"state"'), (3, '0.005*"money" + 0.003*"speaking" + 0.003*"police"')]
  Topic_0 Topic_1     Topic_2   Topic_3
0    kids     car     country     money
1    love  mother   president  speaking
2    city  father       state    police
3   white    love       party      kids
4    high    kids       money     phone
5     job   money        case      case
6   money   music  government     woman
7   black     mom       white       mom
8    guys     god  republican       dad
9    read     war         dad      love

5 0.9
[(0, '0.004*"kids" + 0.003*"city" + 0.002*"white"'), (1, '0.004*"father" + 0.003*"mother" + 0.003*"love"'), (2, '0.003*"president" + 0.003*"country" + 0.002*"party"'), (3, '0.006*"money" + 0.003*"speaking" + 0.002*"police"'), (4, '0.002*"car" + 0.002*"kids" + 0.002*"love"')]
  Topic_0 Topic_1     Topic_2   Topic_3    Topi

In [258]:
min_df = 10
max_df = 0.8
new_models3 = []
for num in [4, 5, 6, 7]:
    ldamodel, output = cluster_fns.new_model_fn(ep_df, 'text', num, min_df, max_df)
    print(cluster_fns.make_topic_df(ldamodel))
    new_models3.append((num, ldamodel))
    print()

10 0.8
[(0, '0.002*"christmas" + 0.002*"white" + 0.002*"chicken"'), (1, '0.004*"mother" + 0.004*"father" + 0.004*"dad"'), (2, '0.003*"state" + 0.003*"government" + 0.003*"president"'), (3, '0.003*"police" + 0.002*"mother" + 0.002*"dad"')]
     Topic_0   Topic_1     Topic_2 Topic_3
0  christmas    mother       state  police
1      white    father  government  mother
2    chicken       dad   president     dad
3        war       mom    speaking     mom
4       book   parents         000  father
5      black      book         war    york
6        mom     black       white   david
7      santa      girl       party  office
8     mother       kid      office  street
9     middle  children      states    town

10 0.8
[(0, '0.003*"christmas" + 0.002*"chicken" + 0.002*"white"'), (1, '0.005*"mother" + 0.004*"father" + 0.004*"dad"'), (2, '0.004*"speaking" + 0.003*"president" + 0.003*"state"'), (3, '0.003*"police" + 0.002*"david" + 0.002*"mother"'), (4, '0.003*"dad" + 0.003*"mom" + 0.002*"black"')

In [255]:
min_df = 10
max_df = 0.9
new_models4 = []
for num in [4, 5, 6, 7]:
    ldamodel, output = cluster_fns.new_model_fn(ep_df, 'text', num, min_df, max_df)
    print(cluster_fns.make_topic_df(ldamodel))
    new_models4.append((num, ldamodel))
    print()

10 0.9
[(0, '0.005*"kids" + 0.003*"white" + 0.003*"black"'), (1, '0.004*"dad" + 0.003*"mom" + 0.003*"love"'), (2, '0.004*"money" + 0.003*"case" + 0.003*"speaking"'), (3, '0.003*"mother" + 0.003*"love" + 0.003*"car"')]
   Topic_0 Topic_1     Topic_2 Topic_3
0     kids     dad       money  mother
1    white     mom        case    love
2    black    love    speaking     car
3    money  father         war  father
4     city  police  government   woman
5  country    kids         job   money
6     high     car     country    kids
7    state   phone         000     job
8     guys   money      mother   music
9    party     kid        guys    book

10 0.9
[(0, '0.006*"kids" + 0.004*"black" + 0.004*"white"'), (1, '0.004*"dad" + 0.003*"father" + 0.003*"love"'), (2, '0.006*"money" + 0.003*"case" + 0.002*"mother"'), (3, '0.003*"mother" + 0.003*"car" + 0.003*"love"'), (4, '0.004*"speaking" + 0.003*"country" + 0.003*"war"')]
   Topic_0 Topic_1     Topic_2 Topic_3     Topic_4
0     kids     dad       

In [259]:
min_df = 5
max_df = 0.7
new_models5 = []
for num in [4, 5, 6, 7]:
    ldamodel, output = cluster_fns.new_model_fn(ep_df, 'text', num, min_df, max_df)
    print(cluster_fns.make_topic_df(ldamodel))
    new_models5.append((num, ldamodel))
    print()

5 0.7
[(0, '0.003*"dad" + 0.003*"mom" + 0.002*"singing"'), (1, '0.003*"police" + 0.003*"speaking" + 0.002*"president"'), (2, '0.002*"mom" + 0.002*"dad" + 0.002*"church"'), (3, '0.002*"mom" + 0.002*"dad" + 0.002*"war"')]
     Topic_0     Topic_1     Topic_2   Topic_3
0        dad      police         mom       mom
1        mom    speaking         dad       dad
2    singing   president      church       war
3    brother  government    speaking   chicken
4       song       party    students      iraq
5       girl         law  government  speaking
6  christmas        town       month   company
7       town         mom        town      girl
8      david       court        bank      food
9       baby  republican     college  hospital

5 0.7
[(0, '0.002*"mom" + 0.002*"dad" + 0.002*"singing"'), (1, '0.003*"police" + 0.003*"speaking" + 0.003*"president"'), (2, '0.003*"students" + 0.002*"church" + 0.002*"speaking"'), (3, '0.002*"mom" + 0.002*"dad" + 0.002*"war"'), (4, '0.003*"dad" + 0.002*"mom" +

In [260]:
min_df = 10
max_df = 0.7
new_models6 = []
for num in [4, 5, 6, 7]:
    ldamodel, output = cluster_fns.new_model_fn(ep_df, 'text', num, min_df, max_df)
    print(cluster_fns.make_topic_df(ldamodel))
    new_models6.append((num, ldamodel))
    print()

10 0.7
[(0, '0.004*"dad" + 0.003*"mom" + 0.002*"police"'), (1, '0.003*"mom" + 0.002*"dad" + 0.002*"girl"'), (2, '0.005*"speaking" + 0.005*"war" + 0.003*"government"'), (3, '0.002*"president" + 0.002*"town" + 0.002*"business"')]
   Topic_0   Topic_1     Topic_2     Topic_3
0      dad       mom    speaking   president
1      mom       dad         war        town
2   police      girl  government    business
3     girl     david        iraq  government
4  brother      song   president       party
5    child   singing     spanish         pay
6    death   brother      states  republican
7     baby     women        town     company
8    women   chicken      police        drug
9      son  speaking      united        bank

10 0.7
[(0, '0.004*"dad" + 0.003*"police" + 0.002*"mom"'), (1, '0.003*"mom" + 0.002*"dad" + 0.002*"chicken"'), (2, '0.005*"speaking" + 0.005*"war" + 0.003*"government"'), (3, '0.002*"government" + 0.002*"president" + 0.002*"business"'), (4, '0.004*"mom" + 0.003*"dad" + 0.002*

In [262]:
# Okay so I think I like 6 topics from this min/max combo
min_df = 15
max_df = 0.65
new_models7 = []
for num in [4, 5, 6, 7]:
    ldamodel, output = cluster_fns.new_model_fn(ep_df, 'text', num, min_df, max_df)
    print(cluster_fns.make_topic_df(ldamodel))
    new_models7.append((num, ldamodel))
    print()

15 0.65
[(0, '0.003*"party" + 0.002*"students" + 0.002*"republican"'), (1, '0.006*"speaking" + 0.004*"war" + 0.003*"police"'), (2, '0.004*"dad" + 0.002*"brother" + 0.002*"baby"'), (3, '0.004*"government" + 0.003*"company" + 0.002*"bank"')]
      Topic_0     Topic_1    Topic_2     Topic_3
0       party    speaking        dad  government
1    students         war    brother     company
2  republican      police       baby        bank
3   president   president  christmas       month
4       women  government        bed         buy
5      police     spanish       food   companies
6     college       court        boy        jobs
7    campaign         law      david        plan
8       trump      states     sister   insurance
9     singing        iraq    chicken     million

15 0.65
[(0, '0.004*"party" + 0.003*"republican" + 0.003*"president"'), (1, '0.005*"speaking" + 0.003*"war" + 0.003*"government"'), (2, '0.004*"dad" + 0.002*"christmas" + 0.002*"brother"'), (3, '0.004*"government" + 0.00

In [263]:
min_df = 20
max_df = 0.65
new_models8 = []
for num in [4, 5, 6, 7]:
    ldamodel, output = cluster_fns.new_model_fn(ep_df, 'text', num, min_df, max_df)
    print(cluster_fns.make_topic_df(ldamodel))
    new_models8.append((num, ldamodel))
    print()

20 0.65
[(0, '0.006*"speaking" + 0.004*"police" + 0.003*"war"'), (1, '0.002*"company" + 0.002*"dad" + 0.002*"women"'), (2, '0.003*"students" + 0.002*"president" + 0.002*"dad"'), (3, '0.003*"dad" + 0.003*"court" + 0.002*"party"')]
      Topic_0     Topic_1     Topic_2     Topic_3
0    speaking     company    students         dad
1      police         dad   president       court
2         war       women         dad       party
3         dad     brother     schools      police
4     spanish         buy         bob  republican
5     chicken        bank     college         gay
6        iraq        baby       girls   president
7       women   christmas       class  government
8  government         bed       month        john
9      states  government  government         law

20 0.65
[(0, '0.007*"speaking" + 0.004*"war" + 0.004*"police"'), (1, '0.002*"company" + 0.002*"women" + 0.002*"dad"'), (2, '0.003*"students" + 0.003*"president" + 0.002*"dad"'), (3, '0.004*"dad" + 0.003*"court" + 0.003*

In [266]:
model6 = new_models7[2][1]
output6 = topic_distribution(by_period, 'text', model6)
output6

{0: [[(0, 0.18767372),
   (1, 0.24076308),
   (2, 0.09844182),
   (3, 0.18870538),
   (4, 0.0587864),
   (5, 0.22562958)]],
 1: [[(0, 0.18563873),
   (1, 0.29880965),
   (2, 0.09997512),
   (3, 0.15308584),
   (4, 0.06434872),
   (5, 0.19814199)]],
 2: [[(0, 0.18271291),
   (1, 0.3171178),
   (2, 0.09323008),
   (3, 0.13908318),
   (4, 0.05060488),
   (5, 0.21725118)]],
 3: [[(0, 0.15722972),
   (1, 0.31456214),
   (2, 0.06289257),
   (3, 0.15033694),
   (4, 0.035542395),
   (5, 0.27943623)]]}

In [277]:
apply_tops_df = pd.DataFrame(columns=['period', 'topic_0', 'topic_1',
                                      'topic_2', 'topic_3', 'topic_4', 'topic_5'])
for key in output6.keys():
    apply_tops_df.loc[len(apply_tops_df)] = [key] + [x[1] for x in output6[key][0]]
apply_tops_df

,period,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5
0,0.0,0.187674,0.240763,0.098442,0.188705,0.058786,0.225630
1,1.0,0.185639,0.298810,0.099975,0.153086,0.064349,0.198142
2,2.0,0.182713,0.317118,0.093230,0.139083,0.050605,0.217251
3,3.0,0.157230,0.314562,0.062893,0.150337,0.035542,0.279436


In [281]:
melted_6 = pd.melt(apply_tops_df, id_vars=['period'])
melted_6.head()

,period,variable,value
0,0.0,topic_0,0.187674
1,1.0,topic_0,0.185639
2,2.0,topic_0,0.182713
3,3.0,topic_0,0.157230
4,0.0,topic_1,0.240763


In [285]:
tops_over_time = alt.Chart(melted_6).mark_line().encode(
    x='period',
    y='value',
    color='variable'
)
tops_over_time.encoding.y.title = 'Percentage'
tops_over_time.title = 'Topic Change Over Time Periods'
tops_over_time

alt.Chart(...)

In [330]:
stack_tops_time = alt.Chart(melted_6).mark_bar().encode(
    alt.X('sum(value):Q',
        scale=alt.Scale(domain=[0, 1])
    ),
    y='period:O',
    color='variable',
    order=alt.Order(
      # Sort the segments of the bars by this field
      'variable:O',
      sort='ascending'
    )
).properties(width=400, height=150).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_title(fontSize=16)
stack_tops_time.encoding.x.title = 'Percentage'
stack_tops_time.title = 'Topic Change Over Time Periods'
stack_tops_time

alt.Chart(...)

Are we interested in any of these?
https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/